In [1]:
from nltk.tokenize import RegexpTokenizer
import pandas as pd
import gensim, logging
import numpy as np

/Users/francesco/miniconda/lib/python2.7/site-packages/pandas/computation/__init__.py:19: UserWarning: The installed version of numexpr 2.4.4 is not supported in pandas and will be not be used

  UserWarning)


In [2]:
logging.basicConfig(format='%(asctime)s : %(Levelname)s : %(message)s', level=logging.INFO)

In [4]:
train_df = pd.read_csv('cikm_training_tweets.txt',header=None)
#test_df = pd.read_csv('cikm_test_tweets.txt',header=None)
#ex_df = pd.read_csv('example_tweets.txt',header=None)

In [5]:
train_df.shape

(3658338, 1)

In [20]:
general_1 = train_df.iloc[0:1200000]

In [21]:
general_2 = train_df.iloc[1200001:2400000]

In [22]:
general_3 = train_df.iloc[2400001:]

In [26]:
general_1.to_csv('general_1.csv',header=False,index=False)

In [27]:
general_2.to_csv('general_2.csv',header=False,index=False)

In [28]:
general_3.to_csv('general_3.csv',header=False,index=False)

In [52]:
def read_df(df):
    df_iter = df.iterrows()
    str2lst = []
    k = 200000
    step = 200000
    for i, row in df_iter:
        #print row[0]
        s = str(row[0])
        tokenizer = RegexpTokenizer('\w+')        
        lst = tokenizer.tokenize(s)
        for e in lst:
            e.decode(encoding='utf-8', errors='ignore')
        if i > k:
            print("iteration: " + str(k))
            k += step
        str2lst.append(lst)
    return str2lst    

In [53]:
sentences = read_df(train_df)

iteration: 200000
iteration: 400000
iteration: 600000
iteration: 800000
iteration: 1000000
iteration: 1200000
iteration: 1400000
iteration: 1600000
iteration: 1800000
iteration: 2000000
iteration: 2200000
iteration: 2400000
iteration: 2600000
iteration: 2800000
iteration: 3000000
iteration: 3200000
iteration: 3400000
iteration: 3600000


In [54]:
k = 1000000
step = 1000000
i = 0
l = []
for lst in sentences:
    i += 1
    #print lst
    if i > k:
        print("iteration: " + str(k))
        k += step
    lst_u = [s.decode('utf-8','ignore') for s in lst]
    l.append(lst_u)
#bigram.add_vocab(lst_u)

iteration: 1000000
iteration: 2000000
iteration: 3000000


In [55]:
bigram = gensim.models.phrases.Phrases(l)

In [56]:
trigram = gensim.models.phrases.Phrases(bigram[l])

In [58]:
bigram.save('cikm_train_bi.txt')

In [59]:
trigram.save('cikm_train_tri.txt')

In [60]:
import random
import time
freq = 10
size_NN = 280
nbr_threads = 5

In [61]:
model = gensim.models.Word2Vec(min_count=freq,size=size_NN, workers=nbr_threads, alpha=0.025, min_alpha=0.025)  # use fixed learning rate
start = time.time()
model.build_vocab(trigram[bigram[l]])
end = time.time()
print("building vocabulary completed in: " + str(end - start))

building vocabulary completed in: 644.048655987


In [62]:
for epoch in range(10):
    print('starting epoch: ' + str(epoch))
    start = time.time()
    random.shuffle(l)
    end = time.time()
    print("shuffling completed in: " + str(end - start))
    start = time.time()
    model.train(trigram[bigram[l]])
    model.save('cikm_train_' + str(epoch) + '.txt')
    end = time.time()
    print("training completed in: " + str(end - start))
    #model.alpha -= 0.002  # decrease the learning rate
    #model.min_alpha = model.alpha  # fix the learning rate, no decay

starting epoch: 0
shuffling completed in: 2.3429350853
training completed in: 4074.53651595
starting epoch: 1
shuffling completed in: 2.27680706978
training completed in: 4012.6652329
starting epoch: 2
shuffling completed in: 2.3128080368
training completed in: 4000.8699441
starting epoch: 3
shuffling completed in: 2.28801107407
training completed in: 4028.26297188
starting epoch: 4
shuffling completed in: 2.31799602509
training completed in: 4000.11492801
starting epoch: 5
shuffling completed in: 2.35216903687
training completed in: 4029.01341414
starting epoch: 6
shuffling completed in: 2.32800197601
training completed in: 3989.63932109
starting epoch: 7
shuffling completed in: 2.3099629879
training completed in: 4024.82909799
starting epoch: 8
shuffling completed in: 2.3010430336
training completed in: 3995.10930204
starting epoch: 9
shuffling completed in: 2.35076117516
training completed in: 4030.05321193


In [63]:
model.save('cikm_training_model.txt')

In [64]:
model.init_sims(replace=True)

In [65]:
model['computer']

array([ 0.10670493, -0.02446322, -0.06322803, -0.05775024, -0.03500049,
       -0.05012653, -0.01246026, -0.02150998,  0.05206715, -0.08702337,
       -0.01110952,  0.00710377, -0.08394822, -0.03541967,  0.06887457,
        0.00813932,  0.07131165, -0.00285849,  0.03620389,  0.05662008,
        0.00154496, -0.0224531 ,  0.03198206,  0.0947962 ,  0.08038258,
       -0.01177219,  0.00952672, -0.03505873, -0.01696639,  0.01818147,
        0.0839735 , -0.02110591,  0.02274374,  0.08800644,  0.00863805,
        0.06114431, -0.06555174,  0.11428583, -0.01764347,  0.00470407,
       -0.11361066, -0.14119987, -0.0130884 ,  0.15760145, -0.03983371,
        0.06915047,  0.08124091,  0.08422206, -0.16422004, -0.02655279,
        0.04046526, -0.0029766 ,  0.10423484,  0.00723427, -0.00080819,
       -0.08020814,  0.02822394,  0.01016829,  0.02648609,  0.075524  ,
       -0.00223604,  0.00904683,  0.04006661,  0.08327244, -0.0036042 ,
        0.00195833,  0.08902995, -0.04331978,  0.07060447, -0.01

In [68]:
 model.most_similar(positive=['computer','apple'], topn=10)

[(u'laptop', 0.6040568351745605),
 (u'macbook', 0.5490165948867798),
 (u'iphone', 0.5324757099151611),
 (u'keyboard', 0.5290461778640747),
 (u'pc', 0.5078140497207642),
 (u'imac', 0.5073572397232056),
 (u'desktop', 0.502971887588501),
 (u'mbp', 0.49664103984832764),
 (u'ipod', 0.49337923526763916),
 (u'dell', 0.48907965421676636)]

In [69]:
model['loook']

array([ -3.41039784e-02,  -4.70374264e-02,  -4.07661945e-02,
         8.40268750e-03,  -9.45763066e-02,   5.17686978e-02,
         1.03190906e-01,  -7.39221200e-02,   1.92335173e-02,
         1.10837325e-01,  -7.20966458e-02,  -3.18825729e-02,
         4.28019166e-02,  -1.29578799e-01,  -2.79977489e-02,
         2.61804014e-02,   1.25758117e-02,  -1.04767084e-03,
         4.40758280e-02,   3.80624644e-02,   3.81986164e-02,
         5.27986139e-02,  -1.92770176e-02,   8.19717124e-02,
         1.80269498e-02,   1.43683022e-02,  -2.26007681e-03,
         1.00035956e-02,   3.25097367e-02,  -5.91108128e-02,
         2.01629046e-02,  -1.21579170e-01,   8.82816128e-03,
        -5.04952669e-03,  -1.12351894e-01,  -3.05208787e-02,
         3.19656059e-02,  -1.38771966e-01,   6.45952746e-02,
         1.25794765e-02,  -9.18958113e-02,  -2.41458956e-02,
         8.28427300e-02,  -5.06904423e-02,  -2.26705242e-02,
         1.18737027e-01,  -7.07202852e-02,   9.09234732e-02,
        -7.44225755e-02,

In [72]:
 model.most_similar(positive=['haha'], topn=10)

[(u'hahaha', 0.6096524596214294),
 (u'lol', 0.5855266451835632),
 (u'hahahaha', 0.5758712887763977),
 (u'hahah', 0.5728620886802673),
 (u'ha', 0.5528308749198914),
 (u'ha_ha', 0.5428324937820435),
 (u'lmao', 0.5292174816131592),
 (u'hah', 0.5279834270477295),
 (u'ahaha', 0.5083060264587402),
 (u'hehe', 0.4958555996417999)]

In [73]:
 model.most_similar(positive=['yay'], topn=10)

[(u'woo_hoo', 0.540040910243988),
 (u'woohoo', 0.539691686630249),
 (u'hooray', 0.5061463117599487),
 (u'yayyy', 0.49870139360427856),
 (u'thank_god', 0.44012898206710815),
 (u'woo', 0.4388554096221924),
 (u'woot', 0.4364403486251831),
 (u'thank_goodness', 0.4350361227989197),
 (u'yayy', 0.43337586522102356),
 (u'yipee', 0.4279119372367859)]

In [74]:
 model.most_similar(positive=['fb'], topn=10)

[(u'facebook', 0.43771177530288696),
 (u'myspace_myspace_com', 0.33800941705703735),
 (u'automate_your_affiliate_marketing', 0.31869468092918396),
 (u'fbook', 0.31814044713974),
 (u'thewebhostinggeek_39_s', 0.30327725410461426),
 (u'our_facebook_page', 0.29117047786712646),
 (u'facebook_founder_mark', 0.283147394657135),
 (u'ftw', 0.2818997800350189),
 (u'pc4pc_comment_truth', 0.28025689721107483),
 (u'socialgood', 0.2778246998786926)]

In [76]:
 model.most_similar(positive=['fir'], topn=10)

[(u'for', 0.2899615466594696),
 (u'fot', 0.26385870575904846),
 (u'adobe_photoshop_cs4', 0.25329649448394775),
 (u'statutory', 0.24410253763198853),
 (u'rockwall', 0.24346378445625305),
 (u'never_happens', 0.2428201586008072),
 (u'complete_virtual', 0.2359333634376526),
 (u'brickhouse', 0.23538696765899658),
 (u'coffee_machine', 0.23466284573078156),
 (u'dodger_game', 0.23070889711380005)]